In [1]:
import os
import pandas as pd
from openai import OpenAI

In [2]:
#OPEN AI KEY

os.environ["OPENAI_API_KEY"] = "SECRET API KEY"

OPENAI_API_KEY: str = os.environ.get("OPENAI_API_KEY")


In [3]:
occupations_list = [
    'ICT%20sales%20professional',
    'Marketing%20manager',
    'Product%20analyst',
    'Product%20manager',
    'Product%20designer',
    'Business%20Intelligence%20professional',
    'Infraestructure%20engineer',
    'Computer%20Systems%20Analyst',
    'Software%20infrastructure%20architect',
    'Web%20developer',
    'Software%20developer',
    'App%20developer',
    'User%20interface%20designer',
    'Software%20engineer',
    'Software%20architect',
    'Software%20Quality%20Assurance%20Analysts%20and%20Testers',
    'Embedded%20systems%20engineer',
    'Web%20and%20Digital%20Interface%20Designers',
    'Database%20infrastructure%20engineer',
    'Network%20architect',
    'Database%20Administrators',
    'Database%20Architects',
    'Network%20and%20Computer%20Systems%20Administrator',
    'Artificial%20intelligence%20engineer',
    'Machine%20Learning%20engineer',
    'Data%20science%20engineer',
    'Data%20analyst',
    'Data%20scientist',
    'Artificial%20Intelligence%20scientist',
    'Data%20architect',
    'ICT%20security%20specialist',
    'IT%20security%20operations',
    'Information%20Security%20Analysts',
    'Product%20Security%20and%20IT%20Security%20Integration%20Specialist',
    'Product%20risk%20specialist',
    'Security%20architect',
    'Database%20support%20engineer',
    'Data%20center%20operations%20engineer',
    'Support%20systems%20engineer',
    'Computer%20Network%20Support%20Specialists'
    ]

# occupations_list = [
#     'Machine%20Learning%20engineer',
#     'Information%20Security%20Analysts',
#     ]

In [5]:
# Read golden KSA's
golden = pd.read_csv("./golden_ksa_v11-junio-2024.csv")

relations_df = pd.DataFrame()
relations_df['source'] = None
relations_df['target'] = None
relations_df['ksao'] = None
relations_df['value'] = None
relations_df['origin'] = None

relations_set = set()


for occ in occupations_list:
    #file_path = f'../GoogleForJobs/MLskillsTAG/infocomm_jsearch_jobposts/3-abril-2024-processed/{occ}_competences_llm.csv'
    file_path = f'../GoogleForJobs/infocomm/4-junio-2024-processed/{occ}_competences_llm.csv'
    if os.path.isfile(file_path) and os.path.getsize(file_path) > 1:

        print(f"Analyzing occupation: {occ}")

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        for x in df["competences_llm"]:
            for item in x.split("\n"):
                # TODO: Checar los casos en que en las competencias no están en lista numérica con punto '.', por ej:
                # 
                # 1. Troubleshooting skills
                # 2. Computer security knowledge
                # 3. High-level experience in:
                #     a. Very high performance/traffic web architectures
                #     b. Configuration management
                #     c. Large scale deployment methods
                #     d. Ansible
                # 
                # 
                #   o también:
                # 
                # - Administration
                # - Maintenance
                # - Technical support
                # - Local area network (LAN)
                # - Connectivity
                # - Departmental host system
                # - Network operating s
                # 
                # 
                # 
                #   o también:
                # 
                # 
                # 4. Cyber Security Software tool proficiency
                #     a. Darktrace
                #     b. EDR
                #     c. Security Operations Center (SOC)
                #     d. SEIM
                # 
                # 
                # 
                # 
                # 
                # Checar: if ". " in item or "- " in item:
                if ". " in item:
                    ksa = item.split(". ")[1]
                    found = False
                    
                    #Busca en "Standard text"
                    # for golden_ksa in golden["Standard text"]:
                    #     if golden_ksa == ksa:
                    #         index = golden.index[golden["Standard text"] == golden_ksa].tolist()[0]
                    #         _ksa = golden.at[index, 'Label']
                    #         record = {'source': occ, 'target': ksa, 'ksao': _ksa,'value': 1, 'origin': 'Standard text'}
                    #         relations_set.add(tuple(record.items()))
                    #         relations_df = pd.concat([relations_df, pd.DataFrame([record])], ignore_index=True)
                    #         found = True
                    #         break
                    
                    if not found:
                        #Busca en "Text"
                        for golden_ksa in golden["Text"]:
                            if golden_ksa == ksa:
                                index = golden.index[golden["Text"] == golden_ksa].tolist()[0]
                                _text = golden.at[index, 'Text']
                                _label = golden.at[index, 'Label']
                                record = {'occ': occ, 'text': _text, 'label': _label, 'value': 1}
                                relations_set.add(tuple(record.items()))
                                relations_df = pd.concat([relations_df, pd.DataFrame([record])], ignore_index=True)
                                found = True
                                break
                    
                    if not found:
                        #Usar nuestro fine-tuned model para obtener clasificación de KSAO y agregar.
                        # Initialize OpenAI client
                        client = OpenAI()

                        # Call OpenAI's completion API with the current text
                        completion = client.chat.completions.create(
                            # Usar el modelo finetuned el 4 de Junio de 2024 id: 9WV7nHBF
                            model="ft:gpt-3.5-turbo-0125:personal::9WV7nHBF",
                            messages=[
                                {"role": "system", "content": "Your job is to identify if the text belongs to knowledge, skill, ability or other based on the KSA taxonomy. Ability is different from skill as it is innate and difficult to measure. You cannot teach ability whereas you can teach skill."},
                                {"role": "user", "content": ksa}
                            ]
                        )
                        # Get the predicted output
                        predicted_output = completion.choices[0].message.content
                        
                        parts = predicted_output.split(':')
                        # If there are two parts and the second part is not empty
                        if len(parts) == 2 and parts[1].strip():
                            # Capitalize the first letter of the second part and strip any leading/trailing whitespace
                            capitalized = parts[1].strip().capitalize()
                        else:
                            capitalized = predicted_output

                        if capitalized in ['Knowledge','Skill','Ability','Other']:
                            record = {'occ': occ, 'text': ksa, 'label': capitalized, 'value': 1}
                            relations_set.add(tuple(record.items()))
                            relations_df = pd.concat([relations_df, pd.DataFrame([record])], ignore_index=True)

                            #Add record to golden KSA to prevent further API calls.
                            new_row = pd.DataFrame({'Label': [capitalized], 'Text': [ksa]})
                            golden_df2 = pd.concat([golden, new_row], ignore_index=True)
                            golden_df2.to_csv("golden_ksa_v11-junio-2024.csv", index=False)
                            golden = pd.read_csv("./golden_ksa_v11-junio-2024.csv")

    else:
        print(f"{occ} - 0")
    

Analyzing occupation: ICT%20sales%20professional
Analyzing occupation: Marketing%20manager
Analyzing occupation: Product%20analyst
Analyzing occupation: Product%20manager
Analyzing occupation: Product%20designer
Analyzing occupation: Business%20Intelligence%20professional
Analyzing occupation: Infraestructure%20engineer
Analyzing occupation: Computer%20Systems%20Analyst
Analyzing occupation: Software%20infrastructure%20architect
Analyzing occupation: Web%20developer
Analyzing occupation: Software%20developer
Analyzing occupation: App%20developer
Analyzing occupation: User%20interface%20designer
Analyzing occupation: Software%20engineer
Analyzing occupation: Software%20architect
Analyzing occupation: Software%20Quality%20Assurance%20Analysts%20and%20Testers
Analyzing occupation: Embedded%20systems%20engineer
Analyzing occupation: Web%20and%20Digital%20Interface%20Designers
Analyzing occupation: Database%20infrastructure%20engineer
Analyzing occupation: Network%20architect
Analyzing occu

In [8]:
grouped_df = relations_df.groupby(['occ', 'text', 'label', 'value']).size().reset_index(name='count')


grouped_df

,occ,text,label,value,count
0,App%20developer,.NET,Knowledge,1,2
1,App%20developer,.NET Core,Knowledge,1,2
2,App%20developer,.NET MVC,Knowledge,1,1
3,App%20developer,12 weeks,Other,1,1
4,App%20developer,2 years of experience with Flutter,Knowledge,1,1
...,...,...,...,...,...
71778,Web%20developer,iOS VoiceOver knowledge,Knowledge,1,1
71779,Web%20developer,iOS and Android platform development,Knowledge,1,1
71780,Web%20developer,jQuery,Knowledge,1,3
71781,Web%20developer,jQuery proficiency,Knowledge,1,1


In [9]:
grouped_df.to_csv('infocomm_jobs_ksao_4-junio-2024.csv', index=False)
